In [1]:
import os
import pandas as pd

In [5]:
language_code = 'de'

filepath = os.path.join(f"../input_files/{language_code}", "uncleaned_sentences.csv")
df = pd.read_csv(filepath, delimiter='\t', header=None)
df.columns = ["id", "sentence"]

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2412215 entries, 0 to 2646182
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   id        int64 
 1   sentence  object
dtypes: int64(1), object(1)
memory usage: 55.2+ MB


In [17]:
df.head()

,id,sentence
0,1,­
1,2,.
2,3,"0,00001% der Stimmrechte (das entspricht 14 St..."
3,4,"0,00001% der Stimmrechte (das entspricht 257 S..."
4,5,00:00 DGAP-AFR: AGROB Immobilien AG: Bekanntma...


In [18]:
# Let's see if there are any duplicates in the dataset
df[df["sentence"].duplicated(keep=False)].sort_values("sentence").head(8)

,id,sentence


In [19]:
df.dtypes

id           int64
sentence    object
dtype: object

In [20]:
# Remove all duplicates from the dataframe
df = df.drop_duplicates("sentence")

In [21]:
lengths: pd.Series = df['sentence'].str.len()
max_characters: int = lengths.max()
max_index = lengths.idxmax()

# Find the sentence with the most characters to see if there are any delimitation issues.
print(f'Longest sentence: {max_characters} characters')
print(df[df.index == max_index]['sentence'].values[0][:600]) # Print out first 600 characters

Longest sentence: 199 characters
06.05.2015 – 17:09 Aalen (ots) - Schrozberg: Reh bei Wildunfall verendet Eine 41 Jahre alte Opel-Fahrerin fuhr am Mitwoch, gegen 5.50 Uhr, auf der Kreisstraße zwischen Kleinbärenweiler und Schöngras.


In [22]:
# Cut off any sentences longer than 200 characters
df = df[df['sentence'].str.len() < 200]

# cut off any sentences with fewer than 40 characters
df = df[df['sentence'].str.len() >= 30]

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2412215 entries, 0 to 2646182
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   id        int64 
 1   sentence  object
dtypes: int64(1), object(1)
memory usage: 55.2+ MB


In [14]:
# Randomly sample n rows to get a reduced dataset for easier training while testing out this method. Set a seed for reproducability.
#n_rows = 30000

#reduced_df = df.sample(n=n_rows, random_state=1)

In [25]:
# Remove id column and save dataframes as csv
df.to_csv(f"../output_files/{language_code}/step0_sentences.csv", sep='\t', index=False)
#reduced_df.to_csv("./french_sentences_reduced.csv", sep='\t', index=False)